## IBM Applied Data Science Capstone Course by Coursera

### Week 5 Final Report

#### Opening a New Cafe in the City of Malmo, Sweden.

#### Malmo is one one of the major cities in Sweden, and coffee or Fika as we call it in Swedish, is a popular part of the local culture. From Artisan coffee shops to age old traditional coffee shops, cafe's are increasingly becoming popular in the city. Being just 20 minutes across the bridge from the Danish Capital city of Copenhagen, there are a lot of toursits visiting the city all year round. The idea of this analysis is to understand where are the different coffee shops acorss malmo today, and cluster the neighbourhoods based on coffee shops and look and compare the data to find the best cluster of neighbourhoods to open a new coffee shop. 

#### Approach:
#### 1. Build a dataframe of neighborhoods in Malmo, Sweden by adding the data from Wikipedia page
#### 2.Get the geographical coordinates of the neighborhoods
#### 3.Obtain the venue data for the neighborhoods from Foursquare API
#### 4. Explore and cluster the neighborhoods
#### 5. Select the best cluster to open a new coffee shop

### Importing Required Libraries 

In [1]:
#!pip install geocoder
#!conda install -c conda-forge folium=0.5.0 --yes
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Malm%C3%B6").text

In [3]:
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
    # create a new DataFrame from the list
ml_df = pd.DataFrame({"Neighborhood": neighborhoodList})

ml_df.head()


,Neighborhood
0,"Almgården, Malmö"
1,Almhög
2,"Almvik, Malmö"
3,"Annelund, Malmö"
4,Annestad


In [7]:
# print the number of rows of the dataframe
ml_df.shape

(276, 1)

### Geocoding: Get the geographical coordinates

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},Malmo, Sweden'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [12]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in ml_df["Neighborhood"].tolist() ]

In [11]:
coords

[[55.57590658997318, 13.059481490329878],
 [55.57742096244874, 13.020957932618408],
 [55.55776407298579, 12.977357953910044],
 [55.58828005594587, 13.025830077485097],
 [58.26667000000003, 12.716670000000022],
 [55.56577505514461, 12.969403756012907],
 [58.077780000000075, 11.666670000000067],
 [55.576297782425, 12.984593116826066],
 [55.57880683668391, 13.02721571809389],
 [55.58519000000007, 12.95604000000003],
 [55.58459000000005, 12.96906000000007],
 [55.582970000000046, 12.996680000000026],
 [55.606886101069556, 13.031442183213487],
 [55.556760000000054, 12.918770000000052],
 [55.59225003961113, 12.981755055012997],
 [55.60107719431454, 13.000865520808082],
 [57.69512000000003, 13.04329000000007],
 [55.57242293023424, 12.961672599726782],
 [55.57340126792263, 13.086056732758479],
 [55.60435859590576, 13.030460875081365],
 [55.59510845201303, 13.034865697513334],
 [55.578419463908176, 13.007217620185532],
 [55.59691004250054, 12.988055061077716],
 [55.58605035706311, 13.00625732236

In [13]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
ml_df['Latitude'] = df_coords['Latitude']
ml_df['Longitude'] = df_coords['Longitude']

In [13]:
print(ml_df.shape)
ml_df

(138, 3)


,Neighborhood,Latitude,Longitude
0,"Almgården, Malmö",55.575907,13.059481
1,Almhög,55.577421,13.020958
2,"Almvik, Malmö",55.557764,12.977358
3,"Annelund, Malmö",55.588280,13.025830
4,Annestad,58.266670,12.716670
5,Annetorp,55.565775,12.969404
6,Apelgården,58.077780,11.666670
7,Ärtholmen,55.576298,12.984593
8,"Augustenborg, Malmö",55.578807,13.027216
9,"Bellevue, Malmö",55.585190,12.956040


In [15]:
ml_df.to_csv("ml_df.csv", index=False)

### Create a map of Malmo City with neighborhoods superimposed on top

In [16]:
address = 'Malmo, Sweden'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Malmo, Sweden {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Malmo, Sweden 55.6052931, 13.0001566.


In [17]:
map_ml = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ml_df['Latitude'], ml_df['Longitude'], ml_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ml)  
    
map_ml

In [18]:
map_ml.save('map_ml.html')

### Use the Foursquare API to explore the neighborhoods

In [48]:
CLIENT_ID = 'CLIENT ID' # your Foursquare ID
CLIENT_SECRET = 'CLIENT SECRET' # your Foursquare Secret
VERSION = 'V' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT ID
CLIENT_SECRET:CLIENT SECRET


### Use API to get  top 100 venues that are within a radius of 2000 meters.

In [20]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(ml_df['Latitude'], ml_df['Longitude'], ml_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(15768, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Almgården, Malmö",55.575907,13.059481,Jump,55.567172,13.062028,Theme Park
1,"Almgården, Malmö",55.575907,13.059481,Falafel N.1,55.583700,13.034183,Falafel Restaurant
2,"Almgården, Malmö",55.575907,13.059481,Jägers Zoo,55.565007,13.067428,Pet Store
3,"Almgården, Malmö",55.575907,13.059481,Systembolaget,55.569909,13.058000,Liquor Store
4,"Almgården, Malmö",55.575907,13.059481,City Gross Malmö,55.586408,13.046537,Supermarket


### Analyzing number of venues that were returned for each neighorhood

In [23]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Almgården, Malmö",76,76,76,76,76,76
Almhög,184,184,184,184,184,184
"Almvik, Malmö",114,114,114,114,114,114
"Annelund, Malmö",184,184,184,184,184,184
Annetorp,92,92,92,92,92,92
Apelgården,10,10,10,10,10,10
"Augustenborg, Malmö",188,188,188,188,188,188
"Bellevue, Malmö",110,110,110,110,110,110
Bellevuegården,132,132,132,132,132,132


In [24]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 186 uniques categories.


### Analyzing Types of Venues 

In [25]:
venues_df['VenueCategory'].unique()[:50]

array(['Theme Park', 'Falafel Restaurant', 'Pet Store', 'Liquor Store',
       'Supermarket', 'Furniture / Home Store', 'Restaurant',
       'Fast Food Restaurant', 'Stadium', 'Bus Stop',
       'Chinese Restaurant', 'Scandinavian Restaurant',
       'Electronics Store', 'Shopping Mall', 'Smoke Shop', 'Playground',
       'Shoe Store', 'Soccer Field', 'Gas Station', 'Convenience Store',
       'Grocery Store', 'Rental Car Location', 'Plaza', 'Flea Market',
       'Park', 'Train Station', 'Skate Park', 'Turkish Restaurant',
       'Food & Drink Shop', 'Sandwich Place', 'Beer Bar',
       'Gym / Fitness Center', 'Coffee Shop', 'Pizza Place',
       'Ice Cream Shop', 'Asian Restaurant', 'Café', 'Cheese Shop',
       'Vietnamese Restaurant', 'Beer Garden', 'Pharmacy',
       'Doner Restaurant', 'Indian Restaurant', 'Cocktail Bar',
       'Thrift / Vintage Store', 'Record Shop', 'Music Venue',
       'Sushi Restaurant', 'Persian Restaurant', 'Greek Restaurant'],
      dtype=object)

In [26]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### Mapping Each Neighbourhood to Venues for Neighbourhood Analysis 

In [27]:
# one hot encoding
ml_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ml_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ml_onehot.columns[-1]] + list(ml_onehot.columns[:-1])
ml_onehot = ml_onehot[fixed_columns]

print(ml_onehot.shape)
ml_onehot.head()

(15768, 187)


,Neighborhoods,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,Bakery,Bar,Baseball Field,Bath House,Beach,Beer Bar,Beer Garden,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Lake,Lighthouse,Lingerie Store,Liquor Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,Nightclub,Office,Opera House,Other Great Outdoors,Other Repair Shop,Outdoor Supply Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Port,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Rest Area,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Soccer Stadium,Soup Place,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Syrian Restaurant,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Almgården, Malmö",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,"Almgården, Malmö",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Almgården, Malmö",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Almgården, Malmö",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Grouping by rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
ml_grouped = ml_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(ml_grouped.shape)
ml_grouped

(137, 187)


,Neighborhoods,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,Bakery,Bar,Baseball Field,Bath House,Beach,Beer Bar,Beer Garden,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Lake,Lighthouse,Lingerie Store,Liquor Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,Nightclub,Office,Opera House,Other Great Outdoors,Other Repair Shop,Outdoor Supply Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Port,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Rest Area,Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Soccer Stadium,Soup Place,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Syrian Restaurant,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Almgården, Malmö",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.026316,0.000000,0.000000,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.052632,0.000000,0.052632,0.00,0.026316,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.00,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.026316,0.000000,0.026316,0.000000,0.00,0.000000,0.026316,0.026316,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.026316,0.00,0.000000,0.078947,0.000000,0.000000,0.026316,0.000000,0.00,0.000000,0.000000,0.000000,0.026316,0.052632,0.000000,0.000000,0.026316,0.026316,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.078947,0.000000,0.000

In [29]:
len(ml_grouped[ml_grouped["Coffee Shop"] > 0])

86

### Create a new DataFrame for Our Objective: Coffee Shops in Malmö

In [31]:
ml_cafe = ml_grouped[["Neighborhoods","Coffee Shop"]]

In [32]:
ml_cafe.head()

,Neighborhoods,Coffee Shop
0,"Almgården, Malmö",0.000000
1,Almhög,0.032609
2,"Almvik, Malmö",0.052632
3,"Annelund, Malmö",0.043478
4,Annetorp,0.043478


### Neighbourhood Clustering: Using K-Means Clustering  Algorithm

#### The Different Neighbourhoods of the City are clustered using K-means clustering, with 3 clusters to be anlaysed as a result. 

In [33]:
kclusters = 3

ml_clustering = ml_cafe.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ml_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1, 2, 2, 0, 2, 2, 0, 2], dtype=int32)

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ml_merged = ml_cafe.copy()

# add clustering labels
ml_merged["Cluster Labels"] = kmeans.labels_

In [35]:
ml_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ml_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,"Almgården, Malmö",0.000000,0
1,Almhög,0.032609,2
2,"Almvik, Malmö",0.052632,1
3,"Annelund, Malmö",0.043478,2
4,Annetorp,0.043478,2


In [41]:
ml_merged = ml_merged.join(ml_df.set_index("Neighborhood"), on="Neighborhood")

print(ml_merged.shape)
ml_merged.head() # check the last columns!

(274, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,"Almgården, Malmö",0.0,0,55.575907,13.059481
0,"Almgården, Malmö",0.0,0,55.575907,13.059481
113,Toftanäs,0.0,0,55.602085,13.094702
113,Toftanäs,0.0,0,55.602085,13.094702
121,Virentofta,0.0,0,55.587795,13.076510


In [42]:
# sort the results by Cluster Labels
print(ml_merged.shape)
ml_merged.sort_values(["Cluster Labels"], inplace=True)
ml_merged

(274, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,"Almgården, Malmö",0.000000,0,55.575907,13.059481
118,Valdemarsro,0.000000,0,55.609526,13.075642
118,Valdemarsro,0.000000,0,55.609526,13.075642
44,Inre hamnen,0.000000,0,55.610414,13.065058
44,Inre hamnen,0.000000,0,55.610414,13.065058
43,"Höja, Malmö",0.000000,0,55.580504,13.065725
119,Videdal,0.000000,0,55.589113,13.072856
119,Videdal,0.000000,0,55.589113,13.072856
110,Södra Sofielund,0.000000,0,56.892240,13.781430
110,Södra Sofielund,0.000000,0,56.892240,13.781430


### Visualizing the Clusters 

In [43]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ml_merged['Latitude'], ml_merged['Longitude'], ml_merged['Neighborhood'], ml_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
map_clusters.save('map_clusters.html')

### Analyzing Each Cluster

#### Our First Cluster consists of neighborhoods that do not have a café , or the frequency/density of coffee shops is low. By Visualizing these neighborhoods on a map, the ones located towards popular city neighborhoods are top contenders for choosing a location for our new Café. These are marked RED in the Map above.

In [44]:
ml_merged.loc[ml_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,"Almgården, Malmö",0.000000,0,55.575907,13.059481
118,Valdemarsro,0.000000,0,55.609526,13.075642
118,Valdemarsro,0.000000,0,55.609526,13.075642
44,Inre hamnen,0.000000,0,55.610414,13.065058
44,Inre hamnen,0.000000,0,55.610414,13.065058
43,"Höja, Malmö",0.000000,0,55.580504,13.065725
119,Videdal,0.000000,0,55.589113,13.072856
119,Videdal,0.000000,0,55.589113,13.072856
110,Södra Sofielund,0.000000,0,56.892240,13.781430
110,Södra Sofielund,0.000000,0,56.892240,13.781430


#### Our second cluster, or the Purple ones in the map, consist of High density of coffee shops/Cafes in the neighborhood. By logical thought, these would be popular tourist and city hotspots. However, this cluster is used as a comparison to the other cluster to identify potential places in the vicinity of these hot neighborhoods that then can be a potential candidate for our café. This means, we can further analyses data by using these as a control group

In [45]:
ml_merged.loc[ml_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
84,Persborg,0.050000,1,55.583020,13.029846
111,"Teatern, Malmö",0.060000,1,55.596690,13.001100
85,Pildammsparken,0.060000,1,55.592040,12.993670
85,Pildammsparken,0.060000,1,55.592040,12.993670
106,Svågertorp,0.050000,1,55.557755,12.994078
106,Svågertorp,0.050000,1,55.557755,12.994078
111,"Teatern, Malmö",0.060000,1,55.596690,13.001100
84,Persborg,0.050000,1,55.583020,13.029846
108,Södervärn,0.050000,1,55.588456,13.011655
92,Rådmansvången,0.060000,1,55.595272,13.000384


#### In our third cluster, these are clusters which have a medium density of cafes . The strategy used here is more penetration to the market. Here we need to also adapt to the offerings available and potential needs and customer requirements. These clusters are also potential neighborhoods that can be considered.

In [46]:
ml_merged.loc[ml_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
20,"Flensburg, Malmö",0.040000,2,55.586050,13.006257
124,Västra Kattarp,0.032967,2,55.581273,13.033005
124,Västra Kattarp,0.032967,2,55.581273,13.033005
129,Ärtholmen,0.032787,2,55.576298,12.984593
129,Ärtholmen,0.032787,2,55.576298,12.984593
127,Västra Söderkulla,0.041667,2,55.571778,13.012395
127,Västra Söderkulla,0.041667,2,55.571778,13.012395
123,Västervång,0.016667,2,55.592165,12.959706
102,"Solbacken, Malmö",0.020000,2,55.588761,12.979245
128,Västra hamnen,0.030000,2,55.614050,12.977400


## Discussion 

#### These Clusters provided a key overview on the initial set of locations considered for opening a café. The next steps then can be to identify more popular hotspots within these neighbourhoods. The parameters for this will then depend on the kind of café being considered. For example, a take away kiosk and an artisan coffee shop have a lot of different parameters to consider. But, the overall analysis done here would surely point out the first step in finding a location using data driven decision making.

### Conclusion

#### Based on the clusters obtained, we can conclude that the RED Clusters are the key areas where we should consider opening the café. Within these we can find location which are in closer proximity to city hotspots, but still have a lower density of coffee shops and avoid increased competition. The Green Spots that are chosen would also be strong contenders, as some of them can be seen in key locations. These are also potential locations for consideration. If these locations are chosen, the onus would be more on the offering that the coffee business is offering, so as to differentiate from the competition.